In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews'

In [ ]:
from dataclasses import dataclass

@dataclass
class DataTransformationConfig:
    root_dir: str
    data_path_train: str
    data_path_test: str
    transformed_token_embedding_path: str
    max_tokens: int
    output_sequence_length: int
    input_dim: int
    output_dim: int
    batch_size: int



    


In [7]:
from sentimentanalyzer.constants import *
from sentimentanalyzer.utils.common import read_yaml, create_directories
from pathlib import Path
import yaml
from typing import Union
from typing import Union
from pathlib import Path
from sentimentanalyzer.utils.common import read_yaml, create_directories  # adjust import as needed
from sentimentanalyzer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH 

In [ ]:

from typing import Union
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Union[str, Path] = CONFIG_FILE_PATH,
        params_filepath: Union[str, Path] = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        print(">>> CONFIG CONTENTS:", self.config)         # debug
        print(">>> CONFIG KEYS:", list(self.config.keys()))
        self.params = read_yaml(params_filepath)
        print(">>> PARAMS CONTENTS:", self.params)         
        print(">>> PARAMS KEYS:", list(self.params.keys()))  

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path_train=config.data_path_train,
            data_path_test = config.data_path_test,
            transformed_token_embedding_path=config.transformed_token_embedding_path,
            max_tokens=self.params.max_tokens,
            output_sequence_length=self.params.output_sequence_length,
            input_dim=self.params.input_dim,
            output_dim=self.params.output_dim,
            batch_size=self.params.batch_size
        )

        return data_transformation_config


In [9]:

from tensorflow.keras.layers import TextVectorization, Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
from sentimentanalyzer.logging import logger
from sentimentanalyzer.constants import *
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf


In [10]:
class DataTransformation:
    def __init__(self, config):
        self.config = config


        self.vectorizer = TextVectorization(
            max_tokens=config.max_tokens,
            output_sequence_length=config.output_sequence_length,
            standardize="lower_and_strip_punctuation",
            split="whitespace"
        )
        self.embedding = Embedding(
            input_dim=config.max_tokens,
            output_dim=config.output_dim,
            mask_zero=True
        )

    def transform(self):
        # Your token-embedding based transformation pipeline
        df_train = pd.read_csv(self.config.data_path_train)[:1000]
        df_test = pd.read_csv(self.config.data_path_test)[:1000]

        train_df, valid_df = train_test_split(df_train, test_size=0.2, random_state=42)
        X_train, X_valid = train_df['text'].astype(str), valid_df['text'].astype(str)
        X_test = df_test['text'].astype(str)

        le = LabelEncoder().fit(train_df['target'])
        y_train, y_valid = le.transform(train_df['target']), le.transform(valid_df['target'])
        y_test = le.transform(df_test['target'])

        # Tokenize & embed
        self.vectorizer.adapt(X_train)
        t_train = self.embedding(self.vectorizer(tf.constant(X_train)))
        t_valid = self.embedding(self.vectorizer(tf.constant(X_valid)))
        t_test = self.embedding(self.vectorizer(tf.constant(X_test)))

        np.save(self.config.transformed_token_embedding_path, t_train.numpy())

        def _ds(X, y):
            return (tf.data.Dataset.from_tensor_slices((X, y))
                    .shuffle(1_000).batch(self.config.batch_size)
                    .prefetch(tf.data.AUTOTUNE))

        train_ds = _ds(t_train, y_train)
        valid_ds = _ds(t_valid, y_valid)
        test_ds = _ds(t_test, y_test)

        os.makedirs(self.config.root_dir, exist_ok=True)
        train_path = os.path.join(self.config.root_dir, "train_ds")
        valid_path = os.path.join(self.config.root_dir, "valid_ds")
        test_path = os.path.join(self.config.root_dir, "test_ds")

        tf.data.experimental.save(train_ds, train_path)
        tf.data.experimental.save(valid_ds, valid_path)
        tf.data.experimental.save(test_ds, test_path)

        print(f"Saved train_ds at {train_path}")
        print(f"Saved valid_ds at {valid_path}")
        print(f"Saved test_ds at {test_path}")

    

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform()
except Exception as e:
    raise RuntimeError("Error occurred during data transformation.") from e


[2025-06-14 22:22:38,919: INFO: common: yaml file: config\config.yaml loaded successfully]
>>> CONFIG CONTENTS: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Spencer0013/NLP-Text-Summarizer-Project/raw/refs/heads/main/Dataa.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_preprocessing': {'root_dir': 'artifacts/data_preprocessing', 'ingestion_dir': 'artifacts/data_ingestion', 'output_dir': 'artifacts/data_preprocessing'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'transformer_data': 'artifacts/data_transformation/transformer_data', 'data_path_train': 'artifacts/data_preprocessing/train_clean.csv', 'model_name': 'bert-base-uncased', 'data_path_test': 'artifacts/data_preprocessing/test_clean.csv', 'transformed_token_embedding_path': 'artifacts/data_transformation/token_embeddings.npy'}, 'model_trainer': {'root_dir': 'artifa